In [1]:
!pip install transformers einops accelerate bitsandbytes

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch 
import base64

In [3]:
checkpoint = "MBZUAI/LaMini-T5-738M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.float32)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:616: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
!pip install langchain langchain-community


In [5]:
from langchain.llms import HuggingFacePipeline

In [6]:
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model=base_model,
        tokenizer=tokenizer,
        max_length=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [7]:
input_prompt = "Write a detailed article about the uses of Lamini 738M in Generative AI"
model = llm_pipeline()
generated_text = model(input_prompt)
generated_text

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Lamini 738M is a powerful generative AI architecture that uses a combination of a deep learning model and a neural network to generate intelligent responses. It is a powerful tool that can be used in a variety of applications, including natural language processing, image recognition, and autonomous vehicles. One of the main uses of Lamini 738M is in Generative AI. It is used in Generative AI to train and optimize machine learning models. It can be used to train models on a wide range of tasks, including natural language processing, image recognition, and autonomous vehicles. It is also'

In [8]:
!pip uninstall -y sagemaker
!pip install sagemaker

Found existing installation: sagemaker 2.224.1
Uninstalling sagemaker-2.224.1:
  Successfully uninstalled sagemaker-2.224.1
  Using cached sagemaker-2.224.1-py3-none-any.whl.metadata (15 kB)
Using cached sagemaker-2.224.1-py3-none-any.whl (1.5 MB)


In [9]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
	'HF_TASK': 'text2text-generation',
    'device_map': 'auto',
    'torch_dtype': 'torch.float32'
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="2.0.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Write a short article on Blockchain",
})

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
------------!

[{'generated_text': 'Blockchain is a decentralized digital ledger that records transactions in a secure and transparent manner. It is built on the principles of cryptography, distributed ledger technology (DLT), and peer-to-peer computing. While digital currencies like Bitcoin have the potential to revolutionize various industries, blockchain remains a relatively new technology, with the introduction of public ledgers like Bitcoin and cryptocurrencies. Additionally, blockchain uses blockchain technology to secure transactions by verifying them with tamp'}]

In [11]:
#hyperparameter payload
prompt= "Write a short article on GenAI"
payload = {
    "inputs":prompt,
    "parameters": {
        "do_sample": True,
        "top_p": 0.7,
        "temperature": 0.3,
        "top_k": 50,
        "max_new_token": 512,
        "repetition_penalty":1.03
        
    }
}


In [12]:
response = predictor.predict(payload)
print(response)

[{'generated_text': 'GenAI is a field of artificial intelligence that focuses on creating intelligent machines that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. GenAI has been around since the early days of AI, but it has since evolved to become a powerful tool for a wide range of applications, including healthcare, finance, and transportation. GenAI algorithms are designed to learn from data, improve accuracy, and make predictions based on'}]


In [13]:
ENDPOINT="huggingface-pytorch-tgi-inference-2024-06-21-14-31-52-224"

In [19]:
import boto3
runtime = boto3.client('runtime.sagemaker')
response = runtime.invoke_endpoint(EndpointName=ENDPOINT, ContentType="application/json", Body=json.dumps(payload)) 

In [20]:
response

{'ResponseMetadata': {'RequestId': '263c97f7-62ef-45c8-b63e-7a0595ac9a8a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '263c97f7-62ef-45c8-b63e-7a0595ac9a8a',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Fri, 21 Jun 2024 14:39:23 GMT',
   'content-type': 'application/json',
   'content-length': '560',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f073d1dcfd0>}

In [21]:
prediction = json.loads(response['Body'].read().decode('utf-8'))
prediction[0]['generated_text']

'GenAI is a field of artificial intelligence that focuses on creating intelligent machines that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. GenAI has been around since the early days of computing, and it has become increasingly popular in recent years. GenAI is a powerful tool that can be used in a variety of applications, including healthcare, finance, and transportation. It can analyze large amounts of data, identify patterns'

In [22]:
import gradio as gr

In [25]:
with gr.Blocks() as demo:
    with gr.Row():
        output_text = gr.Textbox(label="Output")
    with gr.Blocks():
        prompt_text = gr.Textbox(lines=1, label="Prompt")
    with gr.Row():
        submit = gr.Button("Submit")    
    
    def generate_text(prompt):
        payload = {
            "inputs":prompt,
            "parameters": {
                "do_sample": True,
                "top_p": 0.7,
                "temperature": 0.3,
                "top_k": 50,
                "max_new_token": 512,
                "repetition_penalty":1.03
        
            }
        }

        
        response = runtime.invoke_endpoint(EndpointName=ENDPOINT, ContentType="application/json", Body=json.dumps(payload)) 
        prediction = json.loads(response['Body'].read().decode('utf-8'))
        output = prediction[0]['generated_text']
        
        return output
    
    submit.click(
        generate_text,
        inputs=[prompt_text],
        outputs=[output_text]
    )
demo.launch()

Running on local URL:  http://127.0.0.1:7862
Sagemaker notebooks may require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://e072437ec01503673d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
